In [12]:
import numpy as np
import matplotlib.pyplot as plt

# Torch imports
import torch
from torchvision import datasets
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms

# Tensor board!!!
from tensorboardX import SummaryWriter

from IPython import display # A convinient way of plotting 
from tqdm import tqdm

from CIFAR10_experiments import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading data

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor()])

cifar_train = datasets.CIFAR10("data/cifar", train=True, download=True, transform=transform)
cifar_test = datasets.CIFAR10('data/cifar',  train=False, download=True, transform=transform)

cifar_validation = Subset(cifar_test, np.arange(0, len(cifar_test)//2 ))
cifar_test = Subset(cifar_test, np.arange(len(cifar_test)//2, len(cifar_test)))

cifar_loader = DataLoader(cifar_train, batch_size = 32)
cifar_val_loader = DataLoader(cifar_validation, batch_size =32)
cifar_test_loader = DataLoader(cifar_validation, batch_size = 32)

Files already downloaded and verified
Files already downloaded and verified


# Auxiliry functions

In [25]:
criterion = nn.NLLLoss()

def display_plot(arr):
    display.clear_output(wait=True)
    plt.plot(val_array)
    display.display(plt.gcf())
    
def report_result(network, epoch):
    val_acc = get_model_accuracy(network, cifar_val_loader)
    train_acc = get_model_accuracy(network, cifar_loader)
    
    writer.add_scalar('data/tensorboard/validation', val_acc, epoch)
    writer.add_scalar('data/tensorboard/training', train_acc, epoch)

# Training hyperparameters loop

In [ ]:
print("Training Started")

EPOCHS = 20
loss_array = {}
val_array = []
test_array = []

# For Tensorboard.

α = 0.0002
for activation in ["ReLU", "Tanh", "Softplus"]:
    classifier = DynamicClassifier(3, 3, activation)
    classifier.cuda()
    optimizer = optim.Adam(classifier.parameters(), lr = α)
    writer = SummaryWriter("runs/Activation_{}".format(activation))
    
    train(classifier, cifar_loader, criterion, optimizer, writer)
    
writer.close()
print("Training Completed Succesfully!")

# Training Loop

In [26]:
def train(classifier, train_set, crieterion, optimizer, writer):
    for i in tqdm(range(EPOCHS)):
        bulk_loss = 0
        for index, batch in enumerate(train_set):
            batch_size = len(batch[0])
            features = batch[0].cuda()
            labels = batch[1].cuda()

            optimizer.zero_grad()

            prediction = classifier(features)
            loss = criterion(prediction, labels)
            bulk_loss += loss

            if (index%50) == 0 and index != 0:
                writer.add_scalar('data/tensorboard/loss', bulk_loss/50.)
                bulk_loss = 0

            loss.backward()
            optimizer.step()
            
        report_result(classifier, i)

# End of notebook